In [ ]:
import importlib

import torch

In [ ]:
# Test whether progress bars work.
from tqdm.auto import tqdm
_ = list(tqdm(range(1)))

## Prepare dataset

In [ ]:
from awe.data import qa_dataset, swde

_ = importlib.reload(qa_dataset)

In [ ]:
sds = swde.Dataset(suffix='-exact')
pages = [p for w in sds.verticals[0].websites[:1] for p in w.pages]
len(pages)

In [ ]:
#qa_dataset.prepare_entries(pages, skip_existing=True)

In [ ]:
loader = qa_dataset.QaEntryLoader(pages)

In [ ]:
#loader.validate()

In [ ]:
entry = loader[0]
entry.id

In [ ]:
entry.get_all_answer_spans()

In [ ]:
entry.labels

## Invoke Transformer

In [ ]:
from awe import qa_model
importlib.reload(qa_model)
pipeline = qa_model.QaPipeline()

In [ ]:
pipeline.load()

In [ ]:
importlib.reload(qa_dataset)
ds = qa_dataset.QaTorchDataset(pages[:1], pipeline.tokenizer)
len(ds)

In [ ]:
#ds.validate_all()

In [ ]:
from awe import qa_model
importlib.reload(qa_model)
model = qa_model.QaModel(pipeline)

In [ ]:
# Test metric computation.
if False:
    batch = ds.get_encodings(0).convert_to_tensors('pt', prepend_batch_axis=True)
    metrics = model.compute_metrics(batch)
    metrics

In [ ]:
import torch.utils.data
import pytorch_lightning as pl

In [ ]:
from awe import gym
g = gym.Gym(None, None, version_name='')

In [ ]:
trainer = pl.Trainer(
    gpus=torch.cuda.device_count(),
    max_epochs=1,
    logger=g.create_logger(),
)

In [ ]:
import numpy as np
rng = np.random.default_rng(42)

In [ ]:
train_pages = rng.choice(pages, 1000, replace=False)
val_pages = [p for p in pages if p not in train_pages]
len(train_pages), len(val_pages)

In [ ]:
train_ds = qa_dataset.QaTorchDataset(train_pages, pipeline.tokenizer)
val_ds = qa_dataset.QaTorchDataset(val_pages, pipeline.tokenizer)

In [ ]:
train_loader = torch.utils.data.DataLoader(train_ds, batch_size=1, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_ds, batch_size=1)
pred_loader = torch.utils.data.DataLoader(ds, batch_size=1)

In [ ]:
trainer.validate(model, pred_loader)

In [ ]:
preds = trainer.predict(model, pred_loader)

In [ ]:
ds.decode_predictions(preds)

### Finetune

In [ ]:
trainer.fit(model, train_loader)

In [ ]:
trainer.validate(model, pred_loader)

In [ ]:
preds = trainer.predict(model, pred_loader)

In [ ]:
ds.decode_predictions(preds)

### Test on unseen

In [ ]:
test_pages = [p for w in sds.verticals[0].websites[1:2] for p in w.pages[:10]]
len(test_pages)

In [ ]:
test_ds = qa_dataset.QaTorchDataset(test_pages, pipeline.tokenizer)

In [ ]:
#qa_dataset.prepare_entries(test_pages)

In [ ]:
test_loader = torch.utils.data.DataLoader(test_ds, batch_size=1)

In [ ]:
trainer.validate(model, test_loader)

In [ ]:
preds = trainer.predict(model, test_loader)

In [ ]:
ds.decode_predictions(preds[4:10])